# Main Modelling Notebook

In [1]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
from tensorflow.keras import models, layers

import os
import numpy as np
import pandas as pd

from utils.data_load import load_test, load_train
from utils.solution import create_solution

2022-09-23 03:17:28.624148: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-23 03:17:29.073925: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-23 03:17:29.604531: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-09-23 03:17:29.604575: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

----SUCCESS: LOADED DATASET----


In [3]:
# gpu test
tf.config.list_physical_devices('GPU')

2022-09-23 03:22:39.450638: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

2022-09-23 03:22:39.510724: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-23 03:22:39.510923: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


## Load Data

In [2]:
dm = load_train()

dogs 501
Processed 100/500
Processed 200/500
Processed 300/500
Processed 400/500
Processed 500/500
cows 500
Processed 100/500
Processed 200/500
Processed 300/500
Processed 400/500
Processed 500/500
spiders 500
Processed 100/500
Processed 200/500
Processed 300/500
Processed 400/500
Processed 500/500
fishes 500
Processed 100/500
Processed 200/500
Processed 300/500
Processed 400/500
Processed 500/500
butterfiles 500


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


Processed 100/500


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


Processed 200/500


...

Processed 400/500


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


Processed 500/500
horses 500
Processed 100/500
Processed 200/500
Processed 300/500
Processed 400/500
Processed 500/500
chickens 500
Processed 100/500
Processed 200/500
Processed 300/500
Processed 400/500
Processed 500/500
elephants 500
Processed 100/500
Processed 200/500


libpng warning: iCCP: known incorrect sRGB profile


Processed 300/500
Processed 400/500
Processed 500/500
lions 500
Processed 100/500
Processed 200/500
Processed 300/500
Processed 400/500
Processed 500/500
cats 501
Processed 100/500
Processed 200/500
Processed 300/500
Processed 400/500
Processed 500/500
----SUCCESS: LOADED DATASET----


### 0. Default FIT Model

In [14]:
def build_cnn():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3,3), padding='same', activation= 'relu', input_shape=(32,32,3)))
    model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(layers.AveragePooling2D(pool_size=(2, 2), padding='same'))
    model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.AveragePooling2D(pool_size=(2, 2), padding='same'))
    model.add(layers.Flatten())
    model.add(layers.Dense(10, activation='softmax'))
    return model

In [ ]:
model = build_cnn()

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate= 0.001)
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x = dm.X_train, y = dm.y_train, validation_data = (dm.X_valid, dm.y_valid), epochs = 15, batch_size = 32)

In [ ]:
# create_solution(model, name="default-trained")

### 01. Assignment Approach

In [5]:
from utils.models import DefaultModel
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
class Model1(DefaultModel):
    def __init__(self,
                 name='network1',
                 width=32, height=32, depth=3,
                 num_blocks=3,
                 feature_maps=32,
                 num_classes=10, 
                 drop_rate=0.2,
                 batch_norm = True,
                 is_augmentation = False,
                 activation_func='relu',
                 use_skip = True,
                 optimizer='adam',
                 batch_size= 32,
                 num_epochs= 20,
                 learning_rate=0.001,
                 verbose= True, 
                 es_patience = 3):
        super(Model1, self).__init__(name, width, height, depth, num_blocks, feature_maps, num_classes, drop_rate, batch_norm, is_augmentation, 
                                        activation_func, optimizer, batch_size, num_epochs, learning_rate, verbose)
        self.use_skip = use_skip
        self.es_patience = es_patience

    def build_cnn(self):
        """Builds a dynamic CNN based on attributes passed into the class"""
        self.model = models.Model()
        inputs = layers.Input(shape=(self.width, self.height, self.depth))
        h = inputs

        for i in range(self.num_blocks):
            h = self.ResidBlock(h=h, num_channels=self.feature_maps[i])

        h = layers.Flatten()(h)
        h = layers.Dense(units=self.num_classes, activation="softmax")(h)

        self.model = models.Model(inputs=inputs, outputs=h)
        self.model.compile(optimizer=self.optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    def ResidBlock(self, h, num_channels) -> list:
        """Passes input h through a residual block.
        Args:
            h (Matrix): The output of the previous layer
            num_channes (int): The number of channels for our Conv3D layers
        Returns:
            h (Matrix): Output of final.
        """
        h = layers.Conv2D(num_channels, (3,3), padding='same')(h)
        skip_signal = h
        
        if self.batch_norm:
            h = layers.BatchNormalization()(h)   

        h = layers.Activation(self.activation_func)(h)
        h = layers.Conv2D(num_channels, (3,3), padding='same')(h)

        if self.batch_norm:
            h = layers.BatchNormalization()(h)

        if self.use_skip:
            h = skip_signal + h

        h = layers.Activation(self.activation_func)(h)
        h = layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(h)
        h = layers.Dropout(rate=self.drop_rate)(h)

        return h
    
    def fit(self, data_manager, batch_size=None, num_epochs=None, es_patience=None):
        batch_size = self.batch_size if batch_size is None else batch_size
        num_epochs = self.num_epochs if num_epochs is None else num_epochs
        es_patience = self.es_patience if es_patience is None else es_patience
        self.model.compile(optimizer=self.optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        
        early_checkpoint = EarlyStopping(patience=es_patience, monitor='val_accuracy', mode='max')
        callbacks = [early_checkpoint]

        if self.is_augmentation:
            datagen = ImageDataGenerator(horizontal_flip=True, width_shift_range=0.1, height_shift_range=0.1)
            datagen.fit(data_manager.X_train)
            X_train_aug = datagen.flow(data_manager.X_train).x
            
            self.history = self.model.fit(x = X_train_aug, y = data_manager.y_train, validation_data = (data_manager.X_valid, data_manager.y_valid), 
                epochs = num_epochs, batch_size = batch_size, verbose= self.verbose, callbacks=callbacks)

        else:
            self.history = self.model.fit(x = data_manager.X_train, y = data_manager.y_train, validation_data = (data_manager.X_valid, data_manager.y_valid), 
                epochs = num_epochs, batch_size = batch_size, verbose= self.verbose, callbacks=callbacks)

In [7]:
bm1 = Model1()
bm1.build_cnn()
bm1.summary()

2022-09-23 03:22:54.648123: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-23 03:22:54.648928: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-23 03:22:54.649283: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-23 03:22:54.649604: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 32)   896         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 32)  128         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 32, 32, 32)   0           ['batch_normalization[0][0]

In [8]:
bm1.fit(dm, batch_size=32, num_epochs=20)

Epoch 1/20


2022-09-23 03:23:12.311953: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8500
2022-09-23 03:23:23.240206: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


125/125 [==============================] - 22s 8ms/step - loss: 1.9116 - accuracy: 0.3385 - val_loss: 2.2883 - val_accuracy: 0.1660
Epoch 2/20
125/125 [==============================] - 1s 6ms/step - loss: 1.5048 - accuracy: 0.4845 - val_loss: 2.6891 - val_accuracy: 0.2180
Epoch 3/20
125/125 [==============================] - 1s 6ms/step - loss: 1.3043 - accuracy: 0.5543 - val_loss: 1.7104 - val_accuracy: 0.4020
Epoch 4/20
125/125 [==============================] - 1s 6ms/step - loss: 1.1161 - accuracy: 0.6105 - val_loss: 1.8213 - val_accuracy: 0.4280
Epoch 5/20
125/125 [==============================] - 1s 6ms/step - loss: 1.0095 - accuracy: 0.6492 - val_loss: 1.8143 - val_accuracy: 0.4860
Epoch 6/20
125/125 [==============================] - 1s 6ms/step - loss: 0.8871 - accuracy: 0.6995 - val_loss: 1.3258 - val_accuracy: 0.5540
Epoch 7/20
125/125 [==============================] - 1s 6ms/step - loss: 0.7986 - accuracy: 0.7283 - val_loss: 1.6925 - val_accuracy: 0.5060
Epoch 8/20
125/1

In [9]:
bm1.compute_accuracy(dm.X_test, dm.y_test, batch_size=32)

16/16 [==============================] - 0s 2ms/step - loss: 1.4110 - accuracy: 0.6000


0.6000000238418579

In [21]:
create_solution(bm1.model, name="bm1")

722/722 [==============================] - 9s 12ms/step


### 2. Grid Search Model1

In [20]:
from itertools import product


def grid_search_kwargs(grid, obj_class, verbose=True):
    """Self Implemented Grid Search"""
    all_results = []
    for pv in list(product(*grid.values())):
        results = {}
        params = {k:p for p, k in zip(pv, list(grid.keys()))}

        if verbose:
            print(f"INFO | Fitting Model w params: {params}")

        mod = obj_class(**params)
        mod.build_cnn()
        mod.fit(dm)

        if verbose:
            print(f"SUCCESS | Model Fitted")

        results.update(params)

        results['val_accuracy'] = mod.history.history['val_accuracy'][-1]
        results['train_accuracy'] = mod.history.history['accuracy'][-1]
        results['test_accuracy'] = mod.compute_accuracy(dm.X_test, dm.y_test)

        all_results.append(results)


    df = pd.DataFrame(all_results).sort_values('test_accuracy', ascending=False).reset_index()
    print("Best Params:")
    print(df.loc[0])

    return df

grid = {
    "num_blocks": [3,4,5],
    "is_augmentation": [True, False],
    "batch_size": [16, 32],
    "es_patience": [3,6]
}

test_grid = {
    "num_blocks": [3,4],
    "is_augmentation": [True, False],
    "verbose": [False]
}

res = grid_search_kwargs(test_grid, Model1)
res

INFO | Fitting Model w params: {'num_blocks': 3, 'is_augmentation': True, 'verbose': False}
SUCCESS | Model Fitted
8/8 [==============================] - 0s 6ms/step - loss: 1.2878 - accuracy: 0.5980
INFO | Fitting Model w params: {'num_blocks': 3, 'is_augmentation': False, 'verbose': False}
SUCCESS | Model Fitted
8/8 [==============================] - 0s 2ms/step - loss: 1.2365 - accuracy: 0.6400
INFO | Fitting Model w params: {'num_blocks': 4, 'is_augmentation': True, 'verbose': False}
SUCCESS | Model Fitted
8/8 [==============================] - 0s 3ms/step - loss: 1.3783 - accuracy: 0.6060
INFO | Fitting Model w params: {'num_blocks': 4, 'is_augmentation': False, 'verbose': False}
SUCCESS | Model Fitted
8/8 [==============================] - 0s 2ms/step - loss: 1.3579 - accuracy: 0.5960
Best Params:
index                    1
num_blocks               3
is_augmentation      False
verbose              False
val_accuracy         0.674
train_accuracy     0.89075
test_accuracy         0

,index,num_blocks,is_augmentation,verbose,val_accuracy,train_accuracy,test_accuracy
0,1,3,False,False,0.674,0.89075,0.640
1,2,4,True,False,0.616,0.90550,0.606
2,0,3,True,False,0.600,0.83150,0.598
3,3,4,False,False,0.634,0.87750,0.596
